In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import math
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense, Input


def Harmonic_Model():
    input = Input(shape=(1,))
    x = Dense(100, activation="tanh")(input)
    x = Dense(200, activation="tanh")(x)
    x = Dense(200, activation="tanh")(x)
    x = Dense(400, activation="tanh")(x)
    x = Dense(400, activation="tanh")(x)
    x = Dense(400, activation="tanh")(x)
    out = Dense(1, activation=None)(x)
    model = Model(input, out)
    return model


model = Harmonic_Model() #Model creation

x = np.linspace(0, 1, 1000) 
x_tf = tf.convert_to_tensor(x[:, np.newaxis], dtype=tf.float32)
# Initial condition
t_0 = tf.convert_to_tensor([[0.0]], dtype=tf.float32) 
y_0 = tf.convert_to_tensor([[1.0]], dtype=tf.float32)  # Initial position
y_prime_0 = tf.convert_to_tensor([[0.0]], dtype=tf.float32)
def loss_function(x):
    #Gradient with respect to the initial condition
    with tf.GradientTape() as tape:
        tape.watch(t_0)
        model_pred0 = model(t_0)
    grad0 = tape.gradient(model_pred0, t_0)
    
    #Gradient with respect to the points
    with tf.GradientTape() as tape2:
        tape2.watch(x)
        with tf.GradientTape() as tape1:
            tape1.watch(x)
            model_pred = model(x)
        grad1 = tape1.gradient(model_pred, x)
    grad2 = tape2.gradient(grad1, x)
    
    initial_condition_loss = tf.reduce_mean(tf.square((model_pred0 - y_0))) + tf.reduce_mean(tf.square(((grad0 - y_prime_0))))

    physics_loss = tf.reduce_mean(tf.square((grad2 + 4*grad1 + 400*model_pred))) #The loss from the differential equation 

    total_loss = initial_condition_loss + physics_loss
    return total_loss


optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001)
epochs = 10000
for epoch in range(epochs):
    with tf.GradientTape() as final_tape:
        loss = loss_function(x_tf)
    changes = final_tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(changes, model.trainable_variables))

    if epoch % 100 == 0:
        print(f"Epoch {epoch}, Loss: {loss.numpy()}")

        # Prediction and visualization
        t_values = np.linspace(0, 1, 1000).reshape(-1, 1).astype(np.float32)
        t_values_tf = tf.convert_to_tensor(t_values)
        predictions = model(t_values_tf).numpy()
        y = np.exp(-2*t_values)*(np.cos(6*math.sqrt(11)*t_values)+(math.sqrt(11)/33)*np.sin(6*math.sqrt(11)*t_values))
        plt.plot(t_values, predictions, label="Predicted", color="red")
        plt.scatter(t_values, y, s=4)
        plt.xlabel("Time")
        plt.ylabel("Displacement")
        plt.title("Harmonic Oscillation Prediction using PINN")
        plt.legend()
        plt.show()


model.save("harmonic_model.h5")